# Spark Local Test

In [1]:
import pandas as pd
import numpy as np
from zipfile import ZipFile

import configparser
from pyspark.sql import SparkSession, Window
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import udf, col, monotonically_increasing_id, row_number
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
from pyspark.sql.types import TimestampType, DateType
from pyspark.sql import functions as F

from datetime import datetime

import os

## Configure Connection

In [4]:
export PYSPARK_SUBMIT_ARGS = "--master local[2] "

SyntaxError: invalid syntax (<ipython-input-4-5f062b1602f1>, line 1)

In [3]:
configure = SparkConf().setAppName('name').setMaster('loal')
sc = SparkContext(conf = configure)

Exception: Java gateway process exited before sending its port number